In [1]:
import cv2     
import math   
import matplotlib.pyplot as plt   
import pandas as pd
from keras.preprocessing import image   
import numpy as np   
from keras.utils import np_utils
from skimage.transform import resize  

In [3]:
count = 0
videoFile = "/content/drive/MyDrive/video/Tom and jerry.mp4"
cap = cv2.VideoCapture(videoFile)   # capturing the video from the given path
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print (count)
print ("Done!")

298
Done!


In [4]:
test = pd.read_csv('/content/drive/MyDrive/video/test.csv')

In [5]:
test_image = []
for img_name in test.Image_ID:
    img = plt.imread('' + img_name)
    test_image.append(img)
test_img = np.array(test_image)

In [6]:
test_image = []
for i in range(0,test_img.shape[0]):
    a = resize(test_img[i], preserve_range=True, output_shape=(224,224)).astype(int)
    test_image.append(a)
test_image = np.array(test_image)

In [13]:
data = pd.read_csv('/content/drive/MyDrive/video/mapping.csv')     # reading the csv file

In [14]:
X = [ ]     # creating an empty array
for img_name in data.Image_ID:
    img = plt.imread('' + img_name)
    X.append(img)  # storing each image in array X
X = np.array(X)    # converting list to array

In [15]:
y = data.Class
dummy_y = np_utils.to_categorical(y)    # one hot encoding Classes

In [16]:
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224)).astype(int)      # reshaping to 224*224*3
    image.append(a)
X = np.array(image)

In [17]:
from keras.applications.vgg16 import preprocess_input
X = preprocess_input(X)  

In [19]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, dummy_y, test_size=0.3, random_state=42)    # preparing the validation set

In [20]:
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [21]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))    # include_top=False to remove the top layer

58900480/58889256 [==============================] - 1s 0us/step


In [22]:
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)

In [23]:
X_train.shape, X_valid.shape

((208, 7, 7, 512), (90, 7, 7, 512))

In [33]:
X_train = X_train.reshape(208, 7*7*512)   
X_valid = X_valid.reshape(90, 7*7*512)

train = X_train/X_train.max()      
X_valid = X_valid/X_train.max()


model = Sequential()
model.add(InputLayer((7*7*512,)))
model.add(Dense(units=1024, activation='sigmoid'))
model.add(Dense(3, activation='softmax'))    

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1024)              25691136  
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 3075      
Total params: 25,694,211
Trainable params: 25,694,211
Non-trainable params: 0
_________________________________________________________________


In [25]:
# preprocessing the images
test_image = preprocess_input(test_image)

# extracting features from the images using pretrained model
test_image = base_model.predict(test_image)

# converting the images to 1-D form
test_image = test_image.reshape(186, 7*7*512)

# zero centered images
test_image = test_image/test_image.max()

In [39]:
#predictions = model.predict_classes(test_image)
predictions = np.argmax(model.predict(test_image), axis=-1)

In [40]:
print("The screen time of JERRY is", predictions[predictions==1].shape[0], "seconds")
print("The screen time of TOM is", predictions[predictions==2].shape[0], "seconds")

The screen time of JERRY is 0 seconds
The screen time of TOM is 0 seconds
